<a href="https://colab.research.google.com/github/duotien/mono_pitch_tracker/blob/main/mono_pitch_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!nvidia-smi

Tue Feb 28 11:39:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!git clone --recursive https://github.com/duotien/mono_pitch_tracker.git
!apt -qq install -y sox
!pip install pyyaml==5.4.1 sox librosa==0.9.2

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1cKtOEfy014jSjjAlE__o7Pl_LYqXXgsB'
downloaded = drive.CreateFile({'id': file_id})
try:
    print('Downloaded content "{}"'.format(downloaded.GetContentString()))
except OverflowError as e:
    print(f'Download failed, file too large: {e}')
    print(f'Get file from google drive...')
    import shutil
    file_to_copy_path = '/content/drive/MyDrive/DataBase/MedleyDB/V1_MIX_AUDIO_ONLY.zip'
    file_to_paste_path = '/content/V1_MIX_AUDIO_ONLY.zip'

    shutil.copy2(file_to_copy_path, file_to_paste_path)


Download failed, file too large: signed integer is greater than maximum
Get file from google drive...


In [ ]:
%%capture
!unzip -o V1_MIX_AUDIO_ONLY.zip -d /content/mono_pitch_tracker/medleydb/medleydb/data/Audio/

In [ ]:
import sys
import os

NOTEBOOK_DIR = os.getcwd()
MONO_PITCH_TRACKER_DIR = os.path.join(os.getcwd(), 'mono_pitch_tracker/')
MEDLEYDB_DIR = os.path.join(MONO_PITCH_TRACKER_DIR, 'medleydb/')

sys.path.append(MONO_PITCH_TRACKER_DIR)
sys.path.append(MEDLEYDB_DIR)

# Prepare Dataset

In [ ]:
# cut audio into frames
import numpy as np
import pandas as pd

import medleydb
from pitch_tracker.utils.constants import (F_MIN, HOP_LENGTH, N_FFT, N_MELS,
                                           PICKING_FRAME_SIZE,
                                           PICKING_FRAME_STEP,
                                           PICKING_FRAME_TIME, SAMPLE_RATE,
                                           STEP_FRAME, STEP_TIME, WIN_LENGTH,
                                           N_CLASS, )
from pitch_tracker.utils.audio import load_audio_mono
from pitch_tracker.utils.files import get_file_name, list_file_paths_in_dir, list_folder_paths_in_dir

# from pitch_tracker.utils import dataset, files
from pitch_tracker.utils.medleydb_melody import gen_label
from pitch_tracker.utils import dataset

/content/mono_pitch_tracker/medleydb/medleydb/__init__.py:69: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  INST_TAXONOMY = yaml.load(fhandle)
/content/mono_pitch_tracker/medleydb/medleydb/__init__.py:77: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  MIXING_COEFFICIENTS = yaml.load(fhandle)


In [ ]:
%%capture
HOP = 512*5
output_dir = f'{MONO_PITCH_TRACKER_DIR}/content/gen_label/{HOP}/'
AUDIO_PATH = medleydb.AUDIO_PATH

available_tracks = list(list_folder_paths_in_dir(AUDIO_PATH))
available_tracks = [get_file_name(full_path) for full_path in available_tracks]

mtracks = medleydb.load_multitracks(available_tracks)

for mtrack in mtracks:
    gen_label(mtrack.track_id, output_dir, hop=HOP, overwrite=True, convert_to_midi=True, round_method='round', to_csv=True)
    # gen_label(mtrack.track_id, output_dir, hop=HOP, overwrite=True, convert_to_midi=True, round_method='round', to_csv=False)
    pass

In [ ]:
audio_path = f'{MEDLEYDB_DIR}/medleydb/data/Audio/Phoenix_ScotchMorris/Phoenix_ScotchMorris_MIX.wav'
signal, sample_rate = load_audio_mono(audio_path, SAMPLE_RATE, keep_channel_dim=True)
# signal = torch.unsqueeze(signal,0)
# signal = torch.from_numpy(signal)
mel_features = dataset.extract_melspectrogram_feature(signal,N_FFT, HOP_LENGTH, N_MELS, SAMPLE_RATE, backend='librosa')
print(mel_features.shape)

stft_feature = dataset.extract_stft_feature(signal, N_FFT, HOP_LENGTH)
print(stft_feature.shape)

(1, 88, 15258)
(1, 513, 15258)


In [ ]:
csv_path = f'{MONO_PITCH_TRACKER_DIR}/content/gen_label/2560/Melody2_midi/AClassicEducation_NightOwl.csv'
label_dir = f'{MONO_PITCH_TRACKER_DIR}/content/gen_label/2560/Melody2_midi/'
label_dict = dataset.create_label_dict_from_dir(label_dir)
for k, v in label_dict.items():
    print(k, v.shape)

label_dict = dataset.create_label_dict_from_dir(label_dir)
for i,(k,v) in enumerate(label_dict.items()):
    if i==5:
        break
    print(k,v.shape)
    

EthanHein_1930sSynthAndUprightBass (875, 3)
MatthewEntwistle_TheArch (203, 3)
Phoenix_SeanCaughlinsTheScartaglen (1032, 3)
Meaxic_YouListen (1682, 3)
MusicDelta_Punk (170, 3)
Phoenix_ScotchMorris (650, 3)
Schumann_Mignon (1848, 3)
MusicDelta_Beethoven (41, 3)
MusicDelta_Britpop (193, 3)
CroqueMadame_Oil (1120, 3)
LizNelson_Coldwar (1063, 3)
AmarLal_SpringDay1 (158, 3)
MusicDelta_Pachelbel (754, 3)
MusicDelta_Country2 (122, 3)
ChrisJacoby_BoothShotLincoln (893, 3)
BrandonWebster_YesSirICanFly (668, 3)
MusicDelta_Rockabilly (168, 3)
Mozart_BesterJungling (1177, 3)
MusicDelta_ModalJazz (347, 3)
ClaraBerryAndWooldog_TheBadGuys (438, 3)
ClaraBerryAndWooldog_Stella (354, 3)
HeladoNegro_MitadDelMundo (779, 3)
MusicDelta_SwingJazz (259, 3)
MusicDelta_Grunge (288, 3)
AvaLuna_Waterduct (826, 3)
BigTroubles_Phantom (346, 3)
LizNelson_ImComingHome (736, 3)
FacesOnFilm_WaitingForGa (778, 3)
TheDistricts_Vermont (870, 3)
AClassicEducation_NightOwl (581, 3)
MatthewEntwistle_ImpressionsOfSaturn (742, 

In [ ]:
dataset_paths = dataset.create_dataset_path_dict(label_dir)
dataset_paths_df = pd.DataFrame(dataset_paths).transpose()
# dataset_paths_df.columns=['track_id', 'label_path', 'audio_path']
display(dataset_paths_df.head(5))
del dataset_paths_df

/content/mono_pitch_tracker/medleydb/medleydb/multitrack.py:371: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  metadata = yaml.load(f_in)


,0,1
EthanHein_1930sSynthAndUprightBass,/content/mono_pitch_tracker//content/gen_label...,/content/mono_pitch_tracker/medleydb/medleydb/...
MatthewEntwistle_TheArch,/content/mono_pitch_tracker//content/gen_label...,/content/mono_pitch_tracker/medleydb/medleydb/...
Phoenix_SeanCaughlinsTheScartaglen,/content/mono_pitch_tracker//content/gen_label...,/content/mono_pitch_tracker/medleydb/medleydb/...
Meaxic_YouListen,/content/mono_pitch_tracker//content/gen_label...,/content/mono_pitch_tracker/medleydb/medleydb/...
MusicDelta_Punk,/content/mono_pitch_tracker//content/gen_label...,/content/mono_pitch_tracker/medleydb/medleydb/...


In [ ]:
feature_label_gen = dataset.create_feature_label_generator(
    dataset_path_dict=dataset_paths,
    sample_rate=SAMPLE_RATE,
    n_fft=N_FFT,
    n_mels=N_MELS,
    n_class=N_CLASS,
    hop_length=HOP_LENGTH,
    picking_frame_step=PICKING_FRAME_STEP,
    picking_frame_size=PICKING_FRAME_SIZE,
    step_frame=STEP_FRAME,
    step_time=STEP_TIME,
    dist_threshold=0.1,
    empty_threshold=0.3,
)

output_dir = f'{MONO_PITCH_TRACKER_DIR}/content/pickled_database/'
passed_songs = dataset.write_feature_label_to_disk_by_frame(
    feature_label_gen, output_dir, categorize_by_subdir = True)
failed_songs = [label for label in dataset_paths if label not in passed_songs]
display(failed_songs)
del failed_songs

Failed to load audio with `torchaudio`, fallback to `librosa` Failed to process a packet. (Invalid data found when processing input). 


/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


[]

# Modeling & training

In [ ]:
import os
from collections import OrderedDict
from typing import Tuple, Union

import torch
import torchaudio
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import DataLoader, Dataset

from pitch_tracker.utils import dataset, files
from pitch_tracker.utils.constants import (F_MIN, HOP_LENGTH, N_CLASS, N_FFT,
                                           N_MELS, PICKING_FRAME_SIZE,
                                           PICKING_FRAME_STEP,
                                           PICKING_FRAME_TIME, SAMPLE_RATE,
                                           STEP_FRAME, STEP_TIME, WIN_LENGTH)
from pitch_tracker.utils.dataset import AudioDataset
from pitch_tracker.ml.net import NeuralNetwork, train_model, test_model

In [ ]:

DATASET_DIR = f'{MONO_PITCH_TRACKER_DIR}/content/pickled_database/'
SPLIT_BY_DIR = True

if SPLIT_BY_DIR:
    dataset_paths = list(files.list_folder_paths_in_dir(DATASET_DIR))
    train_set, validation_set = train_test_split(dataset_paths, test_size=0.40, random_state=1, shuffle=True)
    validation_set, test_set = train_test_split(validation_set, test_size=0.50, random_state=1, shuffle=True)
    print(f'train_song_set: {len(train_set)}')
    print(f'validation_song_set: {len(validation_set)}')
    print(f'test_song_set: {len(test_set)}')
else:
    dataset_paths = files.list_all_file_paths_in_dir(DATASET_DIR)
    train_set, validation_set = train_test_split(dataset_paths, test_size=0.40, random_state=1, shuffle=True)
    validation_set, test_set = train_test_split(validation_set, test_size=0.50, random_state=1, shuffle=True)
    print(f'train_set: {len(train_set)}')
    print(f'validation_set: {len(validation_set)}')
    print(f'test_set: {len(test_set)}')

train_song_set: 64
validation_song_set: 22
test_song_set: 22


In [ ]:
train_dataset = dataset.AudioDataset(train_set)
validation_dataset = dataset.AudioDataset(validation_set)
test_dataset = dataset.AudioDataset(test_set)
batch_size = 8

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
device = "cuda" if torch.cuda.is_available() \
    else "mps" if torch.backends.mps.is_available() \
    else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
print(model)


NeuralNetwork(
  (conv2d_block1): Sequential(
    (conv2d): Conv2d(1, 256, kernel_size=(3, 3), stride=(1, 1))
    (relu): ReLU()
    (batch_norm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (maxpool2d): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2d_block2): Sequential(
    (conv2d): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
    (relu): ReLU()
    (batch_norm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (maxpool2d): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2d_block3): Sequential(
    (conv2d): Conv2d(256, 210, kernel_size=(3, 3), stride=(1, 1))
    (relu): ReLU()
    (batch_norm): BatchNorm2d(210, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (maxpool2d): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten_layer): Flatten(start_dim=2, end_dim=-1)
  

In [ ]:
# from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter()

In [ ]:
epochs = 50
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_model(model, train_dataloader, loss_fn, optimizer, device)
    test_model(model, validation_dataloader, loss_fn, device)
print("Done!")

Epoch 1
-------------------------------
[    1/  227]  Batch Accuracy: 1.3%, current loss: 13.053930
[   51/  227]  Batch Accuracy: 0.1%, current loss: 12.799308
[  101/  227]  Batch Accuracy: 5.8%, current loss: 12.786565
[  151/  227]  Batch Accuracy: 15.6%, current loss: 12.780796
[  201/  227]  Batch Accuracy: 35.0%, current loss: 12.777308
[  227/  227]  Avg Accuracy: 10.5%, Avg loss: 12.775203
Test Error: 
 Accuracy: 21.7%, Avg loss: 12.762014 

Epoch 2
-------------------------------
[    1/  227]  Batch Accuracy: 25.5%, current loss: 12.753252
[   51/  227]  Batch Accuracy: 27.9%, current loss: 12.746706
[  101/  227]  Batch Accuracy: 22.5%, current loss: 12.744597
[  151/  227]  Batch Accuracy: 42.4%, current loss: 12.743839
[  201/  227]  Batch Accuracy: 16.5%, current loss: 12.743178
[  227/  227]  Avg Accuracy: 26.3%, Avg loss: 12.741026
Test Error: 
 Accuracy: 25.5%, Avg loss: 12.743928 

Epoch 3
-------------------------------
[    1/  227]  Batch Accuracy: 25.7%, current

In [ ]:
import datetime
import pytz

def convert_datetime_timezone(tz1):
    tz1 = pytz.timezone(tz1)
    # tz2 = pytz.timezone(tz2)

    # dt = datetime.datetime.strptime(dt,"%Y-%m-%d %H:%M:%S")
    # dt = tz1.localize(dt)
    timestamp = datetime.datetime.now()
    timestamp = timestamp.astimezone(tz1)
    timestamp = timestamp.strftime("%Y%M%d-%H%M")

    return timestamp

In [ ]:
timestamp = convert_datetime_timezone('Etc/GMT-7')

In [ ]:
model_path = f'/content/drive/MyDrive/School work/KLTN/2022-2023/models/mpt_v01_{timestamp}.pth'
torch.save(model, model_path)


In [ ]:
model = torch.load('/content/drive/MyDrive/School work/KLTN/2022-2023/models/mpt_v01_20231028-2010.pth')

In [ ]:
test_model(model, test_dataloader, loss_fn, device)

Test Error: 
 Accuracy: 35.0%, Avg loss: 16.387311 



In [ ]:
dummy_in = torch.randn(8, 1, 1050, 88).to(device)
in_names = [ "actual_input_1" ] + [ "learned_%d" % i for i in range(16) ]
out_names = [ "output1" ]

torch.onnx.export(model, dummy_in, f="model.onnx", input_names=in_names, output_names=out_names, opset_version=7, verbose=True)